In [39]:
import numpy as np 
import pandas as pd 
import pyreadstat

In [40]:
# reading the metro file
fp3 = "./metro cities/3_LASI_W1_Individual_metrocities_28-03-22.sav"
df_fp3, meta_fp3 = pyreadstat.read_sav(fp3)

In [41]:
# Function to get the variable name given a column label (case-insensitive)
def get_varname_by_label(meta, target_label):
    target_label = target_label.strip().lower()
    for var, label in meta.column_names_to_labels.items():
        if label is not None and label.strip().lower() == target_label:
            return var
    return None

# Define a function to recode yes/no responses (assuming 1=Yes, 2=No)
def recode_yes(x):
    return 1 if x == 1 else 0  # Convert 1 to 1, everything else to 0

##### Definition 2:
Hard threshold: Individuals who have lived in their current city for more than 20 years will not be considered migrants (Binary Variable).

##### Definition 3:
Duration-based categories: Migration status classified into ordinal categories such as <2 years, <5 years, <10 years in the current location. (Ordinal Variable).

Create migration_3 with ordinal groups:
1. 0-2 years -> 1
2. 3-5 years -> 2
3. 6-10 years -> 3
4. 11-20 years -> 4
5. 21-40 years -> 5
6. .>40 years -> 6
7. NaN values -> 0


In [42]:
# params for definition 2
THRESHOLD = 20

# params for definition 3
BINS = [-np.inf, 2, 5, 10, 20, 40, np.inf]
LABELS = [1, 2, 3, 4, 5, 6]

# params for outcome - mental health (CIDI SF score method)
# Define the column names directly (based on the lowercase version of LASI dataset codes)
cols_cidi1 = ["mh204", "mh205", "mh206", "mh207", "mh208", "mh209", "mh210", "mh211"]
cols_cidi2 = ["mh217", "mh218", "mh219", "mh220", "mh221", "mh222"]

# Screening questions
col_MH201 = "mh201"  # Screening for dysphoria
col_MH214 = "mh214"  # Screening for anhedonia

CIDI_1_THRESHOLD = 3 # Depression: if CIDI_1 score >=3, then 1 else 0

In [43]:
# Get variable name for "Since how many years living continuously in this area"
col_name_years = get_varname_by_label(meta_fp3, "Since how many years living continuously in this area")
if col_name_years is None:
    raise ValueError("Column label 'Since how many years living continuously in this area' not found in fp3.sav metadata.")

# Convert the column to numeric (handling errors)
years_living = pd.to_numeric(df_fp3[col_name_years], errors='coerce')

# Create a new DataFrame for migration_2 and migration_3
df_input = pd.DataFrame()

# Create migration_2: 0 if years >= THRESHOLD, else 1
df_input["migration_2"] = np.where(years_living >= THRESHOLD, 0, 1)

# Use pd.cut, and fill NaN values with a default category (e.g., 0 for unknown)
df_input["migration_3"] = pd.cut(years_living, bins=BINS, labels=LABELS, right=True)

# Convert migration_3 to integer, replacing NaN with 0 (or any default category you prefer)
df_input["migration_3"] = df_input["migration_3"].cat.add_categories(0).fillna(0).astype(int)

# Compute CIDI_1 (dysphoria score)
df_fp3["MH201_binary"] = df_fp3[col_MH201].apply(recode_yes)
df_input["CIDI_1"] = df_fp3[cols_cidi1].applymap(recode_yes).sum(axis=1)
df_input.loc[df_fp3["MH201_binary"] == 0, "CIDI_1"] = 0  # Set to 0 if screening was No

# Compute CIDI_2 (anhedonia score)
df_fp3["MH214_binary"] = df_fp3[col_MH214].apply(recode_yes)
df_input["CIDI_2"] = df_fp3[cols_cidi2].applymap(recode_yes).sum(axis=1)
df_input.loc[df_fp3["MH214_binary"] == 0, "CIDI_2"] = 0  # Set to 0 if screening was No

# Create binary depression variable (1 if CIDI_1 >= CIDI_1_THRESHOLD, else 0)
df_input["Depression"] = df_input["CIDI_1"].apply(lambda x: 1 if x >= CIDI_1_THRESHOLD else 0)

# Display first few rows of the new DataFrame
print(df_input.head())


   migration_2  migration_3  CIDI_1  CIDI_2  Depression
0            1            3       0       0           0
1            1            4       0       0           0
2            1            4       0       0           0
3            0            5       0       0           0
4            0            5       0       0           0


C:\Users\siddh\AppData\Local\Temp\ipykernel_6872\4067296259.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_input["CIDI_1"] = df_fp3[cols_cidi1].applymap(recode_yes).sum(axis=1)
C:\Users\siddh\AppData\Local\Temp\ipykernel_6872\4067296259.py:28: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_input["CIDI_2"] = df_fp3[cols_cidi2].applymap(recode_yes).sum(axis=1)


In [44]:
counts = df_input["Depression"].value_counts()
print(counts)

Depression
0    4321
1     152
Name: count, dtype: int64


In [45]:
# df['dm005'] # age
# df['dm008'] #education level
# df['fs318'] # living arrangements
# df['fs323'] # relationship with family and friends -  leave for now
# df['co101']-df['co107'] # comsumption -- household not individual

In [101]:
import pandas as pd

def one_hot_encode(df, column):
    """
    One-hot encodes a categorical column in a DataFrame in-place while retaining the original column.
    The new column names follow the format: col_name_1, col_name_2, ...

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    column (str): The name of the categorical column to encode.

    Returns:
    None (Modifies the DataFrame in-place)
    """
    # Perform one-hot encoding
    dummies = pd.get_dummies(df[column], prefix=column).astype(int)

    # Rename columns to col_name_1, col_name_2, ...
    dummies.columns = [f"{column}_{i+1}" for i in range(dummies.shape[1])]

    # Add the new one-hot encoded columns to the original DataFrame
    for col in dummies.columns:
        df[col] = dummies[col]  # Assign each new column to the original DataFrame

# Example Usage
data = pd.DataFrame({'Category': ['A', 'B', 'A', 'C']})
one_hot_encode(data, 'Category')
print(data)


  Category  Category_1  Category_2  Category_3
0        A           1           0           0
1        B           0           1           0
2        A           1           0           0
3        C           0           0           1


In [102]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

In [103]:
df_fp3['migration_2']=df_input['migration_2']

In [104]:
binary_features=['migration_2']
numerical_features=['dm005']
ordinal_features=['dm008']
categorial_cols=['fs318']
categorial_features=[]
for col in categorial_cols:
    one_hot_encode(df_fp3,col)

    for i in range(df_fp3[col].nunique()):
        categorial_features.append(col+f'_{i+1}')

feature_cols= binary_features + numerical_features + ordinal_features + categorial_features

In [105]:
X=df_fp3[feature_cols]

for col in categorial_features + ordinal_features+ binary_features:
    X.loc[:, col] = X[col].fillna(X[col].mode()[0])
for col in numerical_features:
    X.loc[:, col] = X[col].fillna(X[col].mean())

# X.loc[:, 'fs318'] = X['fs318'].fillna(X['fs318'].mode()[0])
X=sm.add_constant(X)
X.dropna(inplace=True)

In [108]:
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

In [109]:
print(vif_data)

      Variable        VIF
0        const  35.928449
1  migration_2   1.026981
2        dm005   1.200894
3        dm008   1.012105
4      fs318_1   1.018061
5      fs318_2   1.090463
6      fs318_3   1.141120
7      fs318_4   1.002560


In [110]:
# Fit logistic regression model
y = df_input['Depression']
model = sm.Logit(y, X).fit()

# Display model summary
print(model.summary())

         Current function value: 0.147179
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:             Depression   No. Observations:                 4473
Model:                          Logit   Df Residuals:                     4465
Method:                           MLE   Df Model:                            7
Date:                Wed, 19 Mar 2025   Pseudo R-squ.:                0.007705
Time:                        00:43:12   Log-Likelihood:                -658.33
converged:                      False   LL-Null:                       -663.44
Covariance Type:            nonrobust   LLR p-value:                    0.1762
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -3.3521      0.487     -6.888      0.000      -4.306      -2.398
migration_2     0.1401      0.230      0.610      0.542      -

c:\Users\siddh\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [111]:
np.exp(0.1401)

1.1503888319886737